<a href="https://colab.research.google.com/github/itdusty/blood_cells_classification/blob/main/cells_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
from PIL import Image, ImageDraw, ImageFilter
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
os.chdir("/content/drive/MyDrive/Cells classification")

In [33]:
if not os.path.isdir("dataset_1_frame"): #Создаем основной репозиторий
    os.mkdir("dataset_1_frame")
os.chdir("dataset_1_frame") # Указываем используемый репозиторий

# Общие настройки
pack = 50 # Количество папок с изображениями
n_step = 1 # Количество изображений в папке
pix_x = 128 # Размер изображения в пикселях
pix_y = 128
manual = False # При значении True размер и количество клеток устанавливаются вручную
is_blurry = True # Включение размытия
is_noisy = True # Включение шума
delta = 1 # Шаг перемещения клеток
offset = 1100 # Изменение начального номера папки с изображением
debug = False # Отключение создания DataFrame.csv для удобства отладки 
dataframe_available = True # Есть ли файл DataFrame.csv для соединения с новыми данными или нет
dataframe_filename = "./DataFrame1.csv"

# Параметры, действующие при debug = True
sizeR = 1 # Размер точек
sizeG = 3
red_min = 8 # Минимальное количество эритроцитов на изображении
red_max = 20 # Максимальное количество эритроцитов на изображении
gray_min = 5 # Минимальное количество лимфоцитов на изображении
gray_max = 15 # Максимальное количество лимфоцитов на изображении

# Параметры, действующие при debug = False
locR = 1 # Среднее ("центр") нормального распределения для размеров эритроцитов (R) и лейкоцитов (G)
locG = 1.5
scaleR = 0.1 # Стандартное отклонение (разброс или "ширина") нормального распределения для размеров эритроцитов (R) и лейкоцитов (G)
scaleG = 0.1
loc_countR = 50 # Среднее ("центр") нормального распределения для количества эритроцитов (R) и лейкоцитов (G). Произведение со следующими коэффициентами
# определяет количество клеток
loc_countG = 2
scale_countR = 1 # Стандартное отклонение (разброс или "ширина") нормального распределения для количества эритроцитов (R) и лейкоцитов (G)
scale_countG = 1
cells_multiplierR = 100 # Коэффициенты, изменяющие разряд случайного числа. Полученное случайное число будет умножено на соответствующий коэффициент,
# тем самым увеличивая количество эритроцитов (R) и лейкоцитов (G)
cells_multiplierG = 100

# Настройка изображения
color_f = 'white' # Цвет фона изображения
format_image = '.png' # Формат сохраняемого изображения
name_image = 'broun' # Имя изображения
name_files = "traffic" # Имя папки
color_ery = 'red' # Цвет эритроцитов
color_limf = 'grey' # Цвет лимфоцитов
blur_radius = 10
noise_cutoff = 1 # Если случайное значение с нормальным распределением будет выше данного значения - на изображении появится "плохой" пиксель,
# если ниже - без изменений. Для noise_cutoff = 1 вероятность появления шума составляет 0.1587
noise_prob = 0.5 # Вероятность добавления шума: необходима для уменьшения вероятности появления шума.
# При значении 1 появление шума зависит только от noise_cutoff
noise_color = "grey" # Цвет шума: grey или random

ran_coord_rx = [] #Создание используемых массивов
ran_coord_ry = []
ran_coord_gx = []
ran_coord_gy = []
Wxr = []
Wyr = []
Wxg = []
Wyg = []
files = []
erythrocytes = []
lymphocytes = []
noise = int(is_noisy)

for h in range(offset+1, offset+pack+1): # Цикл для создания папок в основном репозитории
    hh = str(h)
    if not os.path.isdir(name_files+hh): #Создание папки
        os.mkdir(name_files+hh)
    os.chdir(name_files+hh) # Название папки
    rep = name_files+hh
    print(os.getcwd()) # Проверка рабочей папки
    if manual:
        ran_red = random.randint(red_min, red_max) #Количество точек в одной папке
        ran_gray = random.randint(gray_min, gray_max)
    else:
        sizeR = round(abs(np.random.normal(loc=locR, scale=scaleR)))
        sizeG = round(abs(np.random.normal(loc=locG, scale=scaleG)))
        ran_red = round(abs(np.random.normal(loc=loc_countR, scale=scale_countR))*cells_multiplierR)
        ran_gray = round(abs(np.random.normal(loc=loc_countG, scale=scale_countG))*cells_multiplierG)
    files.append(rep)
    erythrocytes.append(ran_red)
    lymphocytes.append(ran_gray)
    ran_coord_rx.clear()
    ran_coord_ry.clear()
    ran_coord_gx.clear()
    ran_coord_gy.clear()

    for i in range(0,n_step):
        img = Image.new('RGBA', (pix_x, pix_y), color_f) # Создание изображения
        idraw = ImageDraw.Draw(img) # Создание объекта изображения
        Wxr.clear() # Обнуление списков
        Wyr.clear()
        Wxg.clear()
        Wyg.clear()

        if i == 0: # Генерация начального положения эритроцитов на изображении
            for m in range(0, ran_red):
                ran_coord_rx.append(float(random.uniform(0, pix_x)))
                ran_coord_ry.append(float(random.uniform(0, pix_y)))
        for a in range(0, ran_red): # Цикл для отображения эритроцитов
            ran_coord_rx.append(float(random.uniform((ran_coord_rx[0] - delta), (ran_coord_rx[0] + delta)))) # Определение следующей координаты
            ran_coord_ry.append(float(random.uniform((ran_coord_ry[0] - delta), (ran_coord_ry[0] + delta))))
            ran_coord_rx.remove(ran_coord_rx[0])
            ran_coord_ry.remove(ran_coord_ry[0])
            yi = np.random.choice([1, -1])
            Wxr.append(np.ones(n_step) * ran_coord_rx[0])
            Wyr.append(np.ones(n_step) * ran_coord_ry[0])
            Wxr[0] = Wxr[a-1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyr[0] = Wyr[a - 1] + (yi / np.sqrt(n_step))
            wxr1 = Wxr[0]
            wxr = wxr1[0]
            wyr1 = Wyr[0]
            wyr = wyr1[0]
            xr1 = wxr - sizeR
            yr1 = wyr - sizeR
            xr2 = wxr + sizeR
            yr2 = wyr + sizeR
            idraw.ellipse((xr1, yr1, xr2, yr2), color_ery) # Отображение частицы
            istr = str(i)

        if i == 0: # Генерация начального положения лейкоцитов на изображении
            for z in range(0, ran_gray):
                ran_coord_gx.append(float(random.randint(0, pix_x)))
                ran_coord_gy.append(float(random.randint(0, pix_y)))
        for p in range(0, ran_gray): # Цикл для отображения лейкоцитов
            ran_coord_gx.append(float(random.uniform((ran_coord_gx[0] - delta), (ran_coord_gx[0] + delta)))) # Определение следующей координаты
            ran_coord_gy.append(float(random.uniform((ran_coord_gy[0] - delta), (ran_coord_gy[0] + delta))))
            ran_coord_gx.remove(ran_coord_gx[0])
            ran_coord_gy.remove(ran_coord_gy[0])
            yi = np.random.choice([1, -1])
            Wxg.append(np.ones(n_step) * ran_coord_gx[0])
            Wyg.append(np.ones(n_step) * ran_coord_gy[0])
            Wxg[0] = Wxg[p - 1] + (yi / np.sqrt(n_step)) # Уравнение броуновского движения
            Wyg[0] = Wyg[p - 1] + (yi / np.sqrt(n_step))
            wxg1 = Wxg[0]
            wxg = wxg1[0]
            wyg1 = Wyg[0]
            wyg = wyg1[0]
            xg1 = wxg - sizeG
            yg1 = wyg - sizeG
            xg2 = wxg + sizeG
            yg2 = wyg + sizeG
            idraw.ellipse((xg1, yg1, xg2, yg2), color_limf) # Отображение частицы
            istr = str(i)

        if is_blurry: # При включении размытия применяется Box Blur с радиусом blur_radius
            img = img.filter(ImageFilter.BoxBlur(blur_radius))
            idraw = ImageDraw.Draw(img)
            istr = str(i)

        if is_noisy: # При включении шумов
            output = np.random.normal(size=(pix_x, pix_y))
            # Данные строки создают матрицу размером pix_x и pix_y, состоящую из 0 и 1, где 0 - "нормальный" пиксель, 1 - вероятно "плохой" пиксель
            output[output < noise_cutoff] = 0 
            output[output > noise_cutoff] = 1
            for x in range(pix_x):
                for y in range(pix_y):
                    if output[x][y] == 1:
                        # Если случайное число в диапазоне от 0 до 1 окажется меньше, чем noise_prob, то текущий пиксель является "плохим" (появляется шум)
                        if random.uniform(0, 1) < noise_prob: 
                            if noise_color == "grey":
                                color_part = random.randint(0, 255)
                                color = (color_part, color_part, color_part) # Цифровой шум серого цвета разной интенсивности
                            elif noise_color == "random":
                                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) # Цифровой шум случайного цвета
                            idraw.rectangle(((x, y), (x, y)), color) # Отображение шума
            istr = str(i)

        img.convert("RGB").save(name_image + istr + format_image)
        # if i % 10 == 0:
        #     print(name_image + istr + format_image)
    os.chdir("..") # Возвращение к исходному репозиторию
os.chdir("..")

if not debug:
    if offset and dataframe_available:
        old_frame = pd.read_csv(dataframe_filename, index_col=0)
        mean_rbc = (sum(erythrocytes) + old_frame.erythrocytes.sum()) / (len(erythrocytes) + old_frame.erythrocytes.count())
        mean_wbc = (sum(lymphocytes) + old_frame.lymphocytes.sum()) / (len(lymphocytes) + old_frame.lymphocytes.count())
        old_frame.high_erythrocytes = [1 if old_frame.erythrocytes[i] > mean_rbc else 0 for i in range(old_frame.erythrocytes.count())]
        old_frame.high_lymphocytes = [1 if old_frame.lymphocytes[i] > mean_wbc else 0 for i in range(old_frame.lymphocytes.count())]
    else:
        mean_rbc = sum(erythrocytes)/len(erythrocytes)
        mean_wbc = sum(lymphocytes)/len(lymphocytes)
    high_rbc = [1 if erythrocytes[i]>mean_rbc else 0 for i in range(len(erythrocytes))]
    high_wbc = [1 if lymphocytes[i]>mean_wbc else 0 for i in range(len(lymphocytes))]
    new_frame = pd.DataFrame( {'files' : files, 'rbc' : erythrocytes, 'wbc' : lymphocytes, 'high_rbc': high_rbc, 'high_wbc': high_wbc, 'blur' : blur_radius if is_blurry else 0.0, 'noise' : noise})

    frame = pd.concat([old_frame, new_frame], ignore_index=True) if offset and dataframe_available else new_frame
    print(frame)
    frame.to_csv(dataframe_filename)

d:\Projects\ML\cells\dataset_1_frame\traffic1101
d:\Projects\ML\cells\dataset_1_frame\traffic1102
d:\Projects\ML\cells\dataset_1_frame\traffic1103
d:\Projects\ML\cells\dataset_1_frame\traffic1104
d:\Projects\ML\cells\dataset_1_frame\traffic1105
d:\Projects\ML\cells\dataset_1_frame\traffic1106
d:\Projects\ML\cells\dataset_1_frame\traffic1107
d:\Projects\ML\cells\dataset_1_frame\traffic1108
d:\Projects\ML\cells\dataset_1_frame\traffic1109
d:\Projects\ML\cells\dataset_1_frame\traffic1110
d:\Projects\ML\cells\dataset_1_frame\traffic1111
d:\Projects\ML\cells\dataset_1_frame\traffic1112
d:\Projects\ML\cells\dataset_1_frame\traffic1113
d:\Projects\ML\cells\dataset_1_frame\traffic1114
d:\Projects\ML\cells\dataset_1_frame\traffic1115
d:\Projects\ML\cells\dataset_1_frame\traffic1116
d:\Projects\ML\cells\dataset_1_frame\traffic1117
d:\Projects\ML\cells\dataset_1_frame\traffic1118
d:\Projects\ML\cells\dataset_1_frame\traffic1119
d:\Projects\ML\cells\dataset_1_frame\traffic1120
d:\Projects\ML\cells

In [3]:
os.chdir("dataset")
for folder in os.listdir():
    os.chdir(folder)
    print(folder)
    for file in os.listdir():
        Image.open(file).convert("RGB").save(file)
    os.chdir("..")
os.chdir("..")
print("Done!")

traffic1
traffic10
traffic100
traffic1000
traffic1001
traffic1002
traffic1003
traffic1004
traffic1005
traffic1006
traffic1007
traffic1008
traffic1009
traffic101
traffic1010
traffic1011
traffic1012
traffic1013
traffic1014
traffic1015
traffic1016
traffic1017
traffic1018
traffic1019
traffic102
traffic1020
traffic1021
traffic1022
traffic1023
traffic1024
traffic1025
traffic1026
traffic1027
traffic1028
traffic1029
traffic103
traffic1030
traffic1031
traffic1032
traffic1033
traffic1034
traffic1035
traffic1036
traffic1037
traffic1038
traffic1039
traffic104
traffic1040
traffic1041
traffic1042
traffic1043
traffic1044
traffic1045
traffic1046
traffic1047
traffic1048
traffic1049
traffic105
traffic1050
traffic1051
traffic1052
traffic1053
traffic1054
traffic1055
traffic1056
traffic1057
traffic1058
traffic1059
traffic106
traffic1060
traffic1061
traffic1062
traffic1063
traffic1064
traffic1065
traffic1066
traffic1067
traffic1068
traffic1069
traffic107
traffic1070
traffic1071
traffic1072
traffic1073
traff